In [2]:
import tensorflow_datasets as tfds
import tensorflow as tf
import pandas as pd

In [3]:
df = pd.read_parquet('df_embed.parquet.gzip')

In [4]:
df

,bucketized_user_age,movie_genres,movie_id,movie_title,timestamp,user_gender,user_id,user_occupation_label,user_occupation_text,user_rating,user_zip_code
0,35.0,"[0, 7]",b'3107',b'Backdraft (1991)',977432193,True,b'130',18,b'technician/engineer',5.0,b'50021'
1,25.0,[7],b'2114',"b'Outsiders, The (1983)'",965932967,False,b'3829',0,b'academic/educator',4.0,b'22307'
2,18.0,"[4, 15]",b'256',b'Junior (1994)',1012103552,False,b'1265',21,b'writer',1.0,b'49321'
3,18.0,"[0, 10]",b'1389',b'Jaws 3-D (1983)',972004605,True,b'2896',14,b'sales/marketing',5.0,b'60073'
4,18.0,[0],b'3635',"b'Spy Who Loved Me, The (1977)'",961180111,True,b'5264',17,b'college/grad student',4.0,b'15217'
...,...,...,...,...,...,...,...,...,...,...,...
1000204,25.0,"[4, 7]",b'2290',b'Stardust Memories (1980)',964985140,True,b'4508',21,b'writer',4.0,b'15701'
1000205,25.0,"[0, 1, 15]",b'2643',b'Superman IV: The Quest for Peace (1987)',965335130,True,b'4167',21,b'writer',1.0,b'22032'
1000206,25.0,"[7, 16]",b'593',"b'Silence of the Lambs, The (1991)'",958054732,True,b'5799',3,b'customer service',5.0,b'80916'
1000207,25.0,[7],b'1693',b'Amistad (1997)',968543692,False,b'3224',14,b'sales/marketing',3.0,b'93428'


In [5]:
filtered_data = (
    df.filter(["timestamp", "user_id", "movie_id", "user_rating"])
        .sort_values("timestamp")
        .astype({"user_id": int, "movie_id": int, "user_rating": int})
        .drop(columns=["timestamp"])
)

In [8]:
train = filtered_data.iloc[:900000]
test = filtered_data.iloc[900000:]

In [10]:
train

,user_id,movie_id,user_rating
795357,6040,858,4
285481,6040,2384,4
709440,6040,593,5
152436,6040,1961,4
868294,6040,2019,5
...,...,...,...
953968,26,511,5
513264,26,480,4
631679,48,2968,3
421901,25,2253,4


In [11]:
print(train.query("user_id == 1").shape[0])

0


In [12]:
print(test.query("user_id == 1").shape[0])

53


In [14]:
test.query("user_id == 1")

,user_id,movie_id,user_rating
486815,1,3186,4
985165,1,1721,4
918576,1,1022,5
17504,1,1270,5
52019,1,2340,3
177182,1,1836,5
762275,1,3408,4
313921,1,1207,4
220016,1,2804,5
58354,1,260,4


In [19]:
X_train = train.drop(columns=["user_rating"])
y_train = train["user_rating"]
X_test = test.drop(columns=["user_rating"])
y_test = test["user_rating"]

In [15]:
##BAD BAD BAD
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, r2_score

hgb = HistGradientBoostingRegressor(random_state=0)

In [17]:
type(hgb)

sklearn.ensemble._hist_gradient_boosting.gradient_boosting.HistGradientBoostingRegressor

In [20]:
hgb.fit(X_train, y_train)

HistGradientBoostingRegressor(random_state=0)

In [21]:
print(hgb.score(X_test, y_test))

0.07018701410615702


In [22]:
print(hgb.score(X_train, y_train))

0.09402684518604798


In [23]:
y_pred = hgb.predict(X_test)

In [24]:
y_pred

array([3.54843724, 3.5840094 , 3.49207883, ..., 3.39326735, 3.37404638,
       3.86590447])

In [25]:
print(mean_absolute_error(y_test, y_pred))

0.8508620798953698


Implementation in TensorFlow, Version One

In [26]:
all_users = train["user_id"].unique()
all_movies = train["movie_id"].unique()

In [31]:
all_users

array([6040, 6039, 6038, ...,   27,   26,   25])

In [32]:
all_movies

array([ 858, 2384,  593, ...,  826,  576, 3280])

In [27]:
import tensorflow as tf

In [33]:
user_input = tf.keras.layers.Input(shape=(1,), name="user")
user_as_integer = tf.keras.layers.IntegerLookup(vocabulary=all_users)(user_input)
user_embedding = tf.keras.layers.Embedding(input_dim=len(all_users) + 1, output_dim=32)(user_as_integer)

In [35]:
movie_input = tf.keras.layers.Input(shape=(1,), name="movie")
movie_as_integer = tf.keras.layers.IntegerLookup(vocabulary=all_movies)(movie_input)
movie_embedding = tf.keras.layers.Embedding(input_dim=len(all_movies) + 1, output_dim=32)(movie_as_integer)

In [36]:
dot = tf.keras.layers.Dot(axes=2)([user_embedding, movie_embedding])
flatten = tf.keras.layers.Flatten()(dot)

In [37]:
model = tf.keras.Model(inputs=[user_input, movie_input], outputs=flatten)

model.compile(loss="mse", metrics=[tf.keras.metrics.MeanAbsoluteError()])

In [ ]:
model.fit(
    x={
        "user": X_train["user_id"],
        "movie": X_train["movie_id"]
    },
    y=y_train.values,
    batch_size=256,
    epochs=100,
    validation_split=0.1, # for early stopping
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=1, restore_best_weights=True)
    ],
)


Epoch 1/100
3165/3165 [==============================] - 15s 4ms/step - loss: 8.9111 - mean_absolute_error: 2.6080 - val_loss: 12.1811 - val_mean_absolute_error: 3.1907
Epoch 2/100
3165/3165 [==============================] - 13s 4ms/step - loss: 1.6741 - mean_absolute_error: 0.9783 - val_loss: 11.7479 - val_mean_absolute_error: 3.0751
Epoch 3/100
3165/3165 [==============================] - 12s 4ms/step - loss: 1.0750 - mean_absolute_error: 0.7958 - val_loss: 11.6230 - val_mean_absolute_error: 3.0443
Epoch 4/100
3165/3165 [==============================] - 12s 4ms/step - loss: 0.9382 - mean_absolute_error: 0.7504 - val_loss: 11.5565 - val_mean_absolute_error: 3.0279
Epoch 5/100
2095/3165 [==================>...........] - ETA: 3s - loss: 0.8903 - mean_absolute_error: 0.7329

In [30]:
type(user_input)

keras.engine.keras_tensor.KerasTensor